In [1]:
import numpy as np
import pandas as pd

In [2]:
import pickle

# Load the processed DataFrames from variables.pkl
with open('variables.pkl', 'rb') as f:
    vars_dict = pickle.load(f)

# Extract the DataFrames
customer_df = vars_dict['customer']
date_df = vars_dict['date']
facts_df = vars_dict['facts']
product_df = vars_dict['product']
device_df = vars_dict['device']

# Verify the DataFrames are loaded
print("DataFrames loaded successfully:")
print(f"customer_df shape: {customer_df.shape}")
print(f"date_df shape: {date_df.shape}")
print(f"facts_df shape: {facts_df.shape}")
print(f"product_df shape: {product_df.shape}")
print(f"device_df shape: {device_df.shape}")


DataFrames loaded successfully:
customer_df shape: (1000, 5)
date_df shape: (366, 2)
facts_df shape: (5000, 9)
product_df shape: (25, 4)
device_df shape: (5, 3)


In [4]:
facts_df.head()

,session_id,customer_id,product_id,device_id,date_id,quantity,abandon_date,abandon_time,is_abandoned
0,1,979,20,5,252,4,2023-07-13,2025-12-18 01:09:01,1
1,2,373,12,3,353,2,2023-11-07,2025-12-18 13:40:54,1
2,3,32,3,4,286,2,2023-03-03,2025-12-18 20:37:31,1
3,4,631,25,1,365,2,NaT,NaT,0
4,5,127,9,3,163,3,2023-07-22,2025-12-18 06:41:51,1
